# CSI-EC - TP1 - Exercice 1 - Emmiter

*Warning:* Before running this playbook make sure the following packages are installed.

In [1]:
%pip install ascon
%pip install asyncio

import ascon
import asyncio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


To start we first need to get the seed for generating the key and nonce that will be used by ascon.

In [2]:
key_seed=input("Seed for key > ")
nonce_seed=input("Seed for nonce > ")

After we have the input parameters we need to generate the key and nonce using ascon in XOF mode. 
Additionally, in this step, we configure some extra parameters such as the hash length and the associated data.

In [3]:
hashlength=16
key=ascon.hash(key_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)
nonce=ascon.hash(nonce_seed.encode(),variant="Ascon-Xof", hashlength=hashlength)
associated_data=f'''message_{len(key_seed)*len(nonce_seed)}'''.encode()

We can now verify that the key and nonce is now hashed.

In [4]:
print(f'''Key: {key}
Nonce: {nonce}''')

Key: b'\xf1\x94\xac_\xfdj\xdc-\xd9\xd5D*\x87\xb6\x8en'
Nonce: b'n\x1d\x8d\xe4\x82j\x04]>5\xac\xc7\r^Y\xff'


Having the needed variables we can now create a function that will cipher the messages before sending them according to the key, nonce and associated data defined previously.

In [5]:
def cipher_message(in_message,key, nonce, associated_data):
    try:
        out_message=ascon.encrypt(key, nonce, associated_data, in_message.encode(), variant="Ascon-128")
    except Exception as e:
        print(e)
    return out_message

Now we need a asyncronous function that will await new connections to, once a client is connected, send the encrypted message.
To do so we created a simple loop that will allow to send messages while the client.

In [6]:
async def handle_clients(reader,writer):

    address = writer.get_extra_info('peername')
    print(f"New connection from {address}")
    on=True
    # Send messages
    try:
        while on:
            message=input('> ')
            try:
                out=cipher_message(message,key,nonce,associated_data)
                print(out)
                writer.write(out)
                await writer.drain()

                if message.lower() == 'exit':    
                    on=False
            except:
                continue
    except asyncio.CancelledError:
        print(f'''Connection closed from {address}''')
        pass


To execute the program all is left is a main function that will run the server and expose it to possible receivers.

In [7]:
server = await asyncio.start_server(handle_clients, 'localhost', 8888)
addr = server.sockets[0].getsockname()
print(f'Serving on {addr}')
async with server:
    await server.serve_forever()

Serving on ('127.0.0.1', 8888)


New connection from ('::1', 38868, 0, 0)
b'\xe4\x81\xd9(\xe9\x16k\x7f\x98\xa8\xab@\xc1_k\xd9\xb5,\xd1\xc2:'
b'\xef\x85\xdbd\xef\xdf\x03\xe9\xd3\x96\x90\x8d\xc4S\xbft\x964\xbd\xb0\x8c\xdc\x9dQ\xf9\xa5\x1d\x9e\xfe\x0ec\x8c\x0f \xe7'
b'\xe9\x9c\xdc0A\x05\x1b\x88\x12#\x82m\x93\x9c\x8c\x93\xfbF(\xca'
New connection from ('::1', 46024, 0, 0)
b'\xe4\x81\xd9(\xe9\x16k\x7f\x98\xa8\xab@\xc1_k\xd9\xb5,\xd1\xc2:'
b'\xe9\x9c\xdc0A\x05\x1b\x88\x12#\x82m\x93\x9c\x8c\x93\xfbF(\xca'
New connection from ('::1', 39680, 0, 0)
b'\xe4\x81\xd9(\xe9\x16k\x7f\x98\xa8\xab@\xc1_k\xd9\xb5,\xd1\xc2:'
b'\xef\x85\xdbd\xef\xdf\x03\xe9\xd3\x96\x90\x8d\xc4S\xbft\x964\xa2\xc9}\xfc\xec\xa1\x97U\xbc\x10\xdc\xdb\x80]$\x11\xad%\x9am\x9f'
b'\xe9\x9c\xdc0A\x05\x1b\x88\x12#\x82m\x93\x9c\x8c\x93\xfbF(\xca'
New connection from ('::1', 51798, 0, 0)
b'\xe2\x8b\xc2d\xef\xdf\x14\xfd\xb1p\xf0\x04\tAqJMG\x90\x93\xbd\xea\x8a\xa7\xa4'
b'\xe9\x9c\xdc0A\x05\x1b\x88\x12#\x82m\x93\x9c\x8c\x93\xfbF(\xca'
